In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/media/beans/ssd/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'13zx7s8l'

In [3]:
wandb.init(id='7vjhjgdc', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

13502.597

In [5]:
#m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=3e-5)

In [7]:
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.44 µs


In [9]:
#wandb.watch(m, log="all")

In [10]:
#train_only_parts_of_model(m, part_to_train="fcs_2")

In [11]:
model_stem = "7.13"
START_E = 0

In [ ]:
n_epochs = 100
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = np.interp(e, [0, 48, 120, 150], [3e-4, 3e-4, 3e-5, 1e-5])
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.95091248, 'trn_headings_loss': 0.08371103, 'trn_curvatures_loss': 0.06605256, 'consistency losses/trn_steer_cost': 0.03369678, 'consistency losses/trn_te_loss': 14.95092472, 'consistency losses/trn_torque_delta_loss': 350.83463521, 'logistical/obs_consumed_per_second': 239.62992126, 'logistical/obs_generated_per_second': 106.18110236, 'logistical/data_consumption_ratio': 2.2636346, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_loss': 611.2923689, 'logistical/max_param': 16.28192139, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 234.2421875, 'logistical/obs_generated_per_second': 104.46875, 'logistical/data_consumption_ratio': 2.24019877, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.50349045, 'trn_headings_loss': 0.04425395, 'trn_curvatures_loss': 0.03936768, 'consistency losses/trn_steer_cost': 0.03633511, 'consistency losses/trn_te_loss': 15.22604386, 'consistency losses/trn_torque_loss': 466.96700

{'logistical/obs_consumed_per_second': 238.875, 'logistical/obs_generated_per_second': 113.9453125, 'logistical/data_consumption_ratio': 2.10109622, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.07266557, 'trn_headings_loss': 0.00900161, 'trn_curvatures_loss': 0.01488578, 'consistency losses/trn_steer_cost': 0.02762389, 'consistency losses/trn_te_loss': 16.35738465, 'consistency losses/trn_torque_loss': 139.03574602, 'consistency losses/trn_torque_delta_loss': 130.17424452, 'logistical/max_param': 16.2751236, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 235.8046875, 'logistical/obs_generated_per_second': 102.828125, 'logistical/data_consumption_ratio': 2.30608715, 'logistical/manual_train_pause': 0.00070312, 'trn_control_loss': 0.09559345, 'trn_headings_loss': 0.01102889, 'trn_curvatures_loss': 0.01815444, 'consistency losses/trn_steer_cost': 0.03574932, 'consistency losses/trn_te_loss': 15.02496077, 'consistency losses/trn_torque_delta_loss': 127.34618

{'logistical/obs_consumed_per_second': 187.0, 'logistical/obs_generated_per_second': 108.0, 'logistical/data_consumption_ratio': 1.7341168, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 3

{'trn_control_loss': 0.05827522, 'trn_headings_loss': 0.00718394, 'trn_curvatures_loss': 0.01270288, 'consistency losses/trn_steer_cost': 0.0289439, 'consistency losses/trn_te_loss': 15.86157666, 'consistency losses/trn_torque_delta_loss': 90.30929255, 'logistical/obs_consumed_per_second': 239.69291339, 'logistical/obs_generated_per_second': 104.29133858, 'logistical/data_consumption_ratio': 2.29739807, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_loss': 141.13824286, 'logistical/max_param': 16.2699604, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 233.1796875, 'logistical/obs_generated_per_second': 110.15625, 'logistical/data_consumption_ratio': 2.11924922, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.05903339, 'trn_headings_loss': 0.

In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e35.torch")

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
